In [ ]:
import pandas as pd
import numpy as np
import wget

import nltk
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stop = stopwords.words('english')


import string
from tqdm import tqdm
tqdm.pandas()
from typing import List, Dict

from gensim import models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gurumanikantainnamuri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
reviews = pd.read_csv('project/data/Reviews.csv')

In [ ]:
reviews.shape, reviews.columns

((568454, 10),
 Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
        'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
       dtype='object'))

In [ ]:
reviews.dropna(subset = ['Summary','Text'], inplace = True)
reviews['binary_label'] = [1 if val else 0 for val in reviews['Score'] > 3]

In [ ]:
ps = PorterStemmer()
def clean_text(text) -> List:
    '''
    Given a raw text, this function cleans it by
        1. removing stop words,
        2. punctuations that don't make sense by themselves (:,.!')
            multiple punctuations without spaces are not removed.
        3. stems the words using porter stemmer

    Input: raw text
    output: list of words in clean text
    '''
    words = text.split(' ')
    clean_text = list()

    for word in words:
        if word not in english_stop:
            if word not in string.punctuation:
                word = ps.stem(word)
                clean_text.append(word)

    return clean_text

def get_counts(text, label) -> Dict:
    '''
    Calculates how many times each word in the training dataset occurs in positive(1) and negative(0) classes

    Input:
        text: Array of clean text from training data
        label: Corresponding labels from training data
    Output:
        Dictionary of frequency counts

    '''

    freqs = dict()

    for row in tqdm(zip(text, label)):
        text = row[0]
        label = row[1]

        for word in text:
            key = tuple((word,label))
            try:
                freqs[key] += 1
            except KeyError:
                freqs[key] = 1


    return freqs



In [ ]:
#reviews['clean_summary'] = reviews['Summary'].progress_apply(lambda val: clean_text(val))
reviews['clean_text'] = reviews['Text'].progress_apply(lambda val: clean_text(val))

100%|█████████████████████████████████| 568427/568427 [09:22<00:00, 1010.63it/s]


In [ ]:
#Clean summary
'''useful_data = reviews[['clean_summary','binary_label']].copy(deep = True)
x_train, x_test, y_train, y_test = train_test_split(useful_data['clean_summary'].values,
                                                    useful_data['binary_label'].values,
                                                    test_size = 0.15,
                                                   stratify = useful_data['binary_label'])
'''
# Clean text
useful_data = reviews[['clean_text','binary_label']].copy(deep = True)
x_train, x_test, y_train, y_test = train_test_split(useful_data['clean_text'].values,
                                                    useful_data['binary_label'].values,
                                                    test_size = 0.07)

### Train word2vec embeddings

In [ ]:
wv_cbow = models.Word2Vec(sentences = x_train, min_count = 3, window = 7, vector_size = 50)

In [ ]:
# Always have to be extracted based only on the training data
freq_counts = get_counts(x_train, y_train)

528637it [00:13, 39734.79it/s]


In [ ]:
### Generate sentence embeddings based on the avg. word embeddings from word2vec
#wv = models.keyedvectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary = True)

def get_data_set(x, freqs, wv):
    '''
    Given clean text in an array of lists, this function builds the feature set. For every clean text list
    1. Identify how many times each word occured with positive(1) class and sum the counts of individual words
    2. Identify how many times each word occured with negative(1) class and sum the counts of individual words
    3. Extract word2vec embeddings and generate a mean embedding for the sentence

    Input: Clean dataset -> array/list of clean text split into words
        [ [word1, word2...],
        [word1, word2...],
        .
        .
        [word1, word2...] ]

        freqs: the frequency of words wrt context
        wv: word2vec

    Output: DataFrame with extracted features
    '''

    word_cnt_in_pos_data = list()
    word_cnt_in_neg_data = list()
    embeddings = list()



    for text in tqdm(x, desc = 'Get context word counts and word2vec embeddings'):
        pos_cnt, neg_cnt = 0, 0
        sentence_embd = list()
        for word in text:
            try:
                pos_cnt += freqs[tuple((word,0))]
            except KeyError:
                pos_cnt += 0

            try:
                neg_cnt += freqs[tuple((word,1))]
            except KeyError:
                neg_cnt += 0

            try:
                sentence_embd.append(wv.get_vector(word))
            except KeyError:
                sentence_embd.append([0.000001] * 50) # since the word2vec embeddings generated are 50 dimensional

        word_cnt_in_pos_data.append(pos_cnt)
        word_cnt_in_neg_data.append(neg_cnt)
        embeddings.append(np.mean(sentence_embd, axis = 0))


    data = pd.DataFrame(embeddings)
    data['pos_cnt'] = word_cnt_in_pos_data
    data['neg_cnt'] = word_cnt_in_neg_data



    return data


In [ ]:
train_data = get_data_set(x_train, freq_counts, wv = wv_cbow.wv)
train_data['label'] = y_train

Get context word counts and word2vec embeddings: 100%|█| 528637/528637 [01:17<00


In [ ]:
# Test data for inference
test_data = get_data_set(x_test, freq_counts, wv = wv_cbow.wv)

Get context word counts and word2vec embeddings: 100%|█| 39790/39790 [00:07<00:0


In [ ]:
def get_metrics(pred, y_true):
    accuracy = accuracy_score(pred, y_true)
    precision = precision_score(pred, y_true)
    recall = recall_score(pred, y_true)

    print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}')

    return None

## Models

### Logistic Regression

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(train_data[['pos_cnt','neg_cnt']], train_data['label'])

LogisticRegression()

In [ ]:
pred = log_reg.predict(test_data[['pos_cnt','neg_cnt']])

In [ ]:
accuracy_score(pred, y_test), precision_score(pred, y_test), recall_score(pred, y_test)

(0.8043268108936965, 0.9230769230769231, 0.8405910405754046)

### Decision Tree

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
dtree.fit(train_data[['pos_cnt','neg_cnt']], train_data['label'])

DecisionTreeClassifier()

In [ ]:
pred = dtree.predict(test_data[['pos_cnt','neg_cnt']])

In [ ]:
accuracy_score(pred, y_test), precision_score(pred, y_test), recall_score(pred, y_test)

(0.7794259808660289, 0.8545673076923077, 0.8604399878981718)

### Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split = 7, min_samples_leaf = 3, warm_start = True,
                           verbose = 2, max_features = 'log2')

In [ ]:
rf.fit(train_data[['pos_cnt','neg_cnt']], train_data['label'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   49.1s finished


RandomForestClassifier(max_features='log2', min_samples_leaf=3,
                       min_samples_split=7, verbose=2, warm_start=True)

In [ ]:
pred = rf.predict(test_data[['pos_cnt','neg_cnt']])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


In [ ]:
accuracy_score(pred, y_test), precision_score(pred, y_test), recall_score(pred, y_test)

(0.842894763158772, 0.9402222889756684, 0.8684106222750694)

#### Embeddings and counts
### Logistic Regression

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(train_data.drop(labels = 'label', axis = 1), train_data['label'])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LogisticRegression()

In [ ]:
pred = log_reg.predict(test_data)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
accuracy_score(pred, y_test), precision_score(pred, y_test), recall_score(pred, y_test)

(0.8095501382256849, 0.9197277164140766, 0.8494158116363205)

### Decision Tree

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
dtree.fit(train_data.drop(labels = 'label', axis = 1), train_data['label'])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


DecisionTreeClassifier()

In [ ]:
pred = dtree.predict(test_data)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [ ]:
accuracy_score(pred, y_test), precision_score(pred, y_test), recall_score(pred, y_test)

(0.8532797185222417, 0.9022604675057796, 0.9095617271962193)

### Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split = 4, min_samples_leaf = 3, warm_start = True,
                           verbose = 2, max_features = 'log2')

In [ ]:
rf.fit(train_data.drop(labels = 'label', axis = 1), train_data['label'])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.6min finished


RandomForestClassifier(max_features='log2', min_samples_leaf=3,
                       min_samples_split=4, verbose=2, warm_start=True)

In [ ]:
pred = rf.predict(test_data)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished


In [ ]:
accuracy_score(pred, y_test), precision_score(pred, y_test), recall_score(pred, y_test)

(0.8998994722292033, 0.9811199589005908, 0.9000029454213425)

In [ ]:
train_pred = rf.predict(train_data.drop(labels = 'label', axis = 1))
get_metrics(train_pred, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   13.1s finished


Accuracy: 0.9864803258190403
Precision: 0.9998182404218761
Recall: 0.9831470581226318


## Cross Validation

In [ ]:
def run_cross_validation(estimator, X, y, cv = 5 ):
    best_estimator = cross_validate(estimator, X, y, return_estimator = True, scoring = 'accuracy', cv = cv)

    return best_estimator

In [ ]:
estimator = RandomForestClassifier(n_estimators=100, min_samples_split = 4, min_samples_leaf = 3,
                                   warm_start = True, verbose = 2, max_features = 'log2')
cv_result = run_cross_validation(estimator = estimator,
                               X = train_data.drop(labels='label', axis = 1), y = train_data['label'])

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.5min finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.6s finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.4min finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.6s finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.4min finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.6s finished


In [ ]:
best_estimator_idx = np.argmax(cv_result['test_score'])
pred = cv_result['estimator'][best_estimator_idx].predict(test_data)
get_metrics(pred, y_test)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Accuracy: 0.895727569741141
Precision: 0.9811841767274595
Recall: 0.8955775035901644


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished


## Grid Search

In [ ]:
def run_grid_search(X, y, estimator, param_grid, cv = 5):
    gs = GridSearchCV(estimator = estimator, param_grid = param_grid, scoring = 'accuracy', cv = cv)
    gs_result = gs.fit(X , y)

    return gs_result

In [ ]:
param_grid = {'min_samples_leaf' : [3,5],
                                   'warm_start' : [True, False]}



In [ ]:
gs_result = run_grid_search(X = train_data.drop(labels = 'label', axis = 1), y = train_data['label'],
                estimator = RandomForestClassifier(n_estimators = 100, min_samples_split=4,verbose = 1),
                            param_grid = param_grid)





/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.8min finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.0s finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be r

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.5min finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.9s finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.5min finished
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all stri

In [ ]:
best_estimator = gs_result.best_estimator_


In [ ]:
pred = best_estimator.predict(test_data)
get_metrics(pred, y_test)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy: 0.9010052777079668
Precision: 0.9791292062676599
Recall: 0.9026433413255187


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


In [ ]:
best_estimator

RandomForestClassifier(min_samples_leaf=3, min_samples_split=4, verbose=1,
                       warm_start=True)